In [49]:
import sqlite3

#Connecting to sqlite
conn = sqlite3.connect('DB.sqlite')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE inverted_index")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

Table dropped... 


In [60]:
import sqlite3
import re
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk.tokenize import word_tokenize

class Cleaning:
    def __init__(self,doc):
        self.contents=doc
        self.content=''
        for word in self.contents:
            self.content += word
        #print(self.content)
    def Normalize(self):
        return self.content.lower()

    def Remove_uni(self):

        self.content = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", self.Normalize())
        return self.content 

    def Remove_stopw(self):
        stop=stopwords.words('english')
        remove_stop=self.content=" "" ".join([word for word in word_tokenize(self.Remove_uni()) if word not in (stop)])
        return remove_stop
        
    def Lemma(self):
        lemmatizer = WordNetLemmatizer()
        dum=[]
        list_of_word=word_tokenize(self.Remove_stopw())
        list_of_word.sort()
        for word in list_of_word:
            if word[-2:]=='ed' or word[-3:]=='ing':
                dum.append(lemmatizer.lemmatize(word,'v'))
            else:
                dum.append(lemmatizer.lemmatize(word))
        return dum
    

In [51]:
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize

class InvertedIndex:
    def __init__(self):
        self.conn = sqlite3.connect('DB2.sqlite')
        self.cursor = self.conn.cursor()
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS inverted_index (
                index_id INTEGER PRIMARY KEY AUTOINCREMENT,
                word TEXT NOT NULL,
                frequency INTEGER NOT NULL,
                websiteID INTEGER NOT NULL,
                FOREIGN KEY(websiteID) REFERENCES websites(websiteID)
            )
        """)
    def index_websites(self):
        self.cursor.execute("SELECT websiteID, title, content FROM websites")
        websites = self.cursor.fetchall()
        for website in websites:
            website_id = website[0]
            title = website[1]
            content = website[2]
            # Tokenize
            words = []
            if title:
                title_words = Cleaning(title)
                words += title_words.Lemma()
            if content:
                content_words = Cleaning(content)
                words += content_words.Lemma()

            word_freq = {}
            for word in words:
                if word in word_freq:
                    word_freq[word] += 1
                else:
                    word_freq[word] = 1
            for word, frequency in word_freq.items():
                self.cursor.execute("INSERT INTO inverted_index (word, websiteID, frequency) VALUES (?, ?, ?)", (word, website_id, frequency))
        self.conn.commit()
        print('finished')

In [52]:
inv=InvertedIndex()
inv.index_websites()

finished


In [59]:
a=Cleaning("Hello, his name is Preme. He's looks for a book that includes a love story. (love books) 1000 baht 120")
#a.Remove_uni()
#a.Remove_stopw()
print(a.Lemma())


['1000', '120', 'baht', 'book', 'book', 'hello', 'he', 'includes', 'look', 'love', 'love', 'name', 'preme', 'story']
